In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd
import  os 

#### 加载原始数据进行训练

In [ ]:
filepath_list = []
file_dir = "../netflix-prize-data/savedata"
for  file_name in  os.listdir(  file_dir) :
    temppath  =   os.path.join(  os.path.abspath(file_dir)  ,file_name   ) 
    filepath_list.append(temppath )


#### 1,1488844,3,2005-09-06
####  iid , uid , score , date
####  load_from_file 是可以不给timestamp, 没有就是None
####   load_from_df 不能给timestamp ,否者报错

In [ ]:
# 读入训练文件的数量
N =10 

data = pd.DataFrame()
for path in filepath_list[:N]:
    df1 = pd.read_csv(path, header=None)
    df1.columns = ['uid', 'iid', 'rating' ,'date']
    df1= df1.loc[:,['iid', 'uid', 'rating']]
    data = pd.concat([data, df1] , axis  = 0 )

In [ ]:
print("开始读入数据....")
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(data, reader=reader)
train_set = data.build_full_trainset()
print("数据加载成功....")

#### 使用SGD算法进行优化

In [ ]:
bsl_options = {'method': 'sgd', 'n_epochs': 5}
print("als 算法启动....")
algo = BaselineOnly(bsl_options=bsl_options)

#### 定义K折交叉验证迭代器，K=3 ， 并计算rmse

In [ ]:

kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)


In [ ]:
#  预测probe数据集
# F:\rs\L5\netflix-prize-data\probegoodX

basedir  ="../netflix-prize-data/probegoodX"
newfilepath_list =[] 
for  file_name in  os.listdir(  basedir) :
    temppath  =   os.path.join(  os.path.abspath(basedir)  ,file_name   ) 
    newfilepath_list.append(temppath )
    
#排序    
sorted( newfilepath_list) 


In [ ]:
# NN 控制测试集中文件的格式
NN = 10  
testsets  = pd.DataFrame()
for j in  range(NN):
    probe_csv_path =newfilepath_list[j]
    probedata = pd.read_table(  probe_csv_path ,delimiter= "," ,skiprows=1 , header =None )
    probedata.columns = ["user","item","rating","timestamp"]
    probedata  = probedata.loc[: ,["user","item","rating"] ]
    testsets = pd.concat(   [testsets   , probedata ]  ,axis  =0  )


In [ ]:
reader = Reader(line_format='user item rating', sep=',')
probedata= Dataset.load_from_df(testsets, reader=reader)
predictions = algo.test(testsets.values)
accuracy.rmse(predictions, verbose=True)